# RL for Autonomous Vehicle Tasks: Safety and Traffic Optimization

## Agents

In [15]:
# TODO: define agents here

## Scenarios

In [16]:
import gymnasium
import highway_env
from matplotlib import pyplot as plt
%matplotlib inline
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
import logging
import json
import os
import numpy as np
import tensorboard

### Lane Changing

In [17]:
lane_changing_env = gymnasium.make('highway-v0', render_mode='rgb_array')
config = {
    "vehicles_count": 100,  # Number of vehicles in the environment
    "controlled_vehicles": 1,  # Number of vehicles controlled by the agent
    "duration": 30,  # Duration of each episode
    #"reverse_penalty": 0,  # Large penalty for reversing
    # "drifting_penalty": -1,  # Penalty for drifting or sharp turns
    # "off_road_penalty": -1,  # Penalize for going off-road
    "collision_reward": -1,  # Stronger penalty for collisions
    "lane_change_reward": 0.1,  # Reward for staying in lane
    "right_lane_reward": 0.2,  # Reward for being in the right lane
    "high_speed_reward": 0.4,  # Reward for high speed
    "ego_spacing": 1.5,  # Desired minimum spacing between the ego vehicle and the vehicle in front
    "reward_speed_range": [20, 30],  # Reward only within this speed range
    "simulation_frequency": 5,  # Lower simulation speed to avoid erratic behavior
    "initial_lane_id": None,
    "policy_frequency": 1,  # Fewer policy updates per second
    "screen_width": 600,
    "screen_height": 400,
    "offscreen_rendering": False,
    "offroad_terminal": True,
    "render_agent": True,
    "show_trajectories": True,
    "action": {
        "type": "DiscreteAction"  # Discrete control (steer left, right, accelerate)
    },
    "observation": {
        "type": "Kinematics"
    },
}

lane_changing_env.unwrapped.configure(config)
lane_changing_env.reset()

(array([[ 1.        ,  0.6696173 ,  0.25      ,  0.3125    ,  0.        ],
        [ 1.        ,  0.10771749,  0.25      , -0.04923636,  0.        ],
        [ 1.        ,  0.20912096,  0.25      , -0.01369097,  0.        ],
        [ 1.        ,  0.3164464 ,  0.5       , -0.02589392,  0.        ],
        [ 1.        ,  0.4215841 ,  0.5       , -0.04881948,  0.        ]],
       dtype=float32),
 {'speed': 25,
  'crashed': False,
  'action': 7,
  'rewards': {'collision_reward': 0.0,
   'right_lane_reward': 0.3333333333333333,
   'high_speed_reward': 0.5,
   'on_road_reward': 1.0}})

### Roundabout

In [18]:
roundabout_env = gymnasium.make('roundabout-v0', render_mode='rgb_array')

roundabout_config = {
    "controlled_vehicles": 1,  # One agent-controlled vehicle
    "duration": 11,  # Duration of each episode
    "collision_reward": -1,  # Strong penalty for collisions
    "high_speed_reward": 0.2,  # Incentivize maintaining appropriate speed
    "lane_change_reward": -0.05,  # Minor reward for lane changes
    "screen_width": 600,
    "screen_height": 400,
    "offscreen_rendering": False,
    "offroad_terminal": True,
    "render_agent": True,
    "show_trajectories": True,
    "ego_spacing": 1.0,  # Minimum spacing between vehicles
    "incoming_vehicle_destination": None,  # All vehicles depart from the same lane
    "action": {
        "type": "DiscreteAction",  # Discrete control for ease of navigation
        "target_speeds": [0, 8, 16],  # Two target speeds for the agent
    },
    "observation": {
        "type": "Kinematics",  # Focus on vehicle dynamics
        "absolute": True,  # Use absolute coordinates
        "features_range": {
            "x": [-100, 100],  # Maximum visible range in meters
            "y": [-100, 100],
            "vx": [-15, 15],  # Maximum velocity range in m/s
            "vy": [-15, 15],
        },
    }
}

roundabout_env.unwrapped.configure(roundabout_config)
roundabout_env.reset()

(array([[ 1.0000000e+00,  2.0000000e-02,  4.4999999e-01,  0.0000000e+00,
         -5.3333336e-01],
        [ 1.0000000e+00, -2.4032215e-02,  1.9855088e-01,  1.0000000e+00,
          1.5310200e-01],
        [ 1.0000000e+00, -2.0917192e-01,  1.1767375e-01,  5.4431701e-01,
          9.6755505e-01],
        [ 1.0000000e+00,  1.0000000e+00, -2.0000000e-02, -1.0000000e+00,
          2.2204460e-16],
        [ 1.0000000e+00, -1.7556924e-01, -9.5788546e-02, -5.3577900e-01,
          9.8202038e-01]], dtype=float32),
 {'speed': 8,
  'crashed': False,
  'action': 1,
  'rewards': {'collision_reward': False,
   'high_speed_reward': 0.0,
   'lane_change_reward': False,
   'on_road_reward': True}})

### Overtaking

In [19]:
overtaking_env = gymnasium.make('highway-v0', render_mode='rgb_array')
config = {
    "vehicles_count": 50,  # Number of vehicles in the environment
    "controlled_vehicles": 1,  # Number of vehicles controlled by the agent
    "duration": 30,  # Duration of each episode
    "collision_reward": -1,  # Stronger penalty for collisions
    "lane_change_reward": 0.5,  # Reward for staying in lane
    "right_lane_reward": 0.1,  # Reward for being in the right lane
    "overtaking_reward": 1,  # Reward for overtaking
    "reward_speed_range": [20, 30],  # Reward only within this speed range
    "simulation_frequency": 5,  # Lower simulation speed to avoid erratic behavior
    "policy_frequency": 1,  # Fewer policy updates per second
    "ego_spacing": 1.5,  # Desired minimum spacing between the ego vehicle and the vehicle in front
    "screen_width": 600,
    "screen_height": 400,
    "offscreen_rendering": False,
    "offroad_terminal": True,
    "show_trajectories": True,
    "render_agent": True,
    "action": {
        "type": "DiscreteAction"  # Discrete control (steer left, right, accelerate)
    },
    "observation": {
        "type": "Kinematics"
    },
}
overtaking_env.unwrapped.configure(config)
overtaking_env.reset()

(array([[ 1.        ,  0.6860827 ,  0.        ,  0.3125    ,  0.        ],
        [ 1.        ,  0.11369688,  0.        , -0.01974093,  0.        ],
        [ 1.        ,  0.22093678,  0.5       , -0.01566802,  0.        ],
        [ 1.        ,  0.31923932,  0.75      , -0.02453135,  0.        ],
        [ 1.        ,  0.42891288,  0.25      , -0.03623617,  0.        ]],
       dtype=float32),
 {'speed': 25,
  'crashed': False,
  'action': 6,
  'rewards': {'collision_reward': 0.0,
   'right_lane_reward': 0.0,
   'high_speed_reward': 0.5,
   'on_road_reward': 1.0}})

## Training and Testing

In [20]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def create_directory(path):
    """
    Create directory if it doesn't exist
    
    Args:
        path (str): Directory path to create
    """
    try:
        os.makedirs(path, exist_ok=True)
        logger.info(f"Directory created: {path}")
    except Exception as e:
        logger.error(f"Error creating directory {path}: {e}")

def save_pipeline_data(agent_name, stage, environment_name, data):
    """
    Save pipeline stage data
    
    Args:
        agent_name (str): Name of the RL agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        data (dict): Data to be saved
    """
    create_directory(f"results/{agent_name}")
    
    try:
        filename = f"results/{agent_name}/{environment_name}_{stage}_data.json"
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        logger.info(f"Data saved for {stage} in {environment_name}")
    except Exception as e:
        logger.error(f"Error saving data: {e}")

In [29]:
def train(agent, environment, agent_name, stage, environment_name, timesteps=1000):
    """
    Train the agent in a specific environment and save the model
    
    Args:
        agent (sb3.BaseAlgorithm): RL agent to train
        environment (gym.Env): Environment to train in
        agent_name (str): Name of the agent
        stage (str): Current pipeline stage
        environment_name (str): Name of the environment
        timesteps (int, optional): Number of training timesteps. Defaults to 100.
    
    Returns:
        Trained agent
    """
    try:
        # Wrap environment with Monitor
        log_dir = f"logs/{agent_name}"
        os.makedirs(log_dir, exist_ok=True)
        env = Monitor(environment, log_dir)
            
        # Set environment and learn
        agent.set_env(env)
        agent.learn(total_timesteps=timesteps)

        # Get training results
        training_results = {
            'rewards': env.get_episode_rewards(),
        }
        
        # Create directories for model and results
        create_directory(f"models/{agent_name}")
        
        # Save model
        agent.save(f"models/{agent_name}/{environment_name}_model")
        
        # Prepare and save training information
        training_info = {
            'agent_name': agent_name,
            'environment_name': environment_name,
            'stage': stage,
            'total_timesteps': timesteps,
            'training_results': training_results
        }
        save_pipeline_data(agent_name, "train", environment_name, training_info)
        
        logger.info(f"Training completed for {agent_name} in {environment_name}")
        return agent
    
    except Exception as e:
        logger.error(f"Training failed for {agent_name} in {environment_name}: {e}")
        raise



In [22]:
def test(agent, environment, agent_name, stage, environment_name, num_episodes=10):
    test_results = []
    total_rewards = []
    total_collisions = 0
    traffic_speeds = []

    for episode in range(num_episodes):
        try:
            obs, info = environment.reset()
            done = truncated = False
            episode_reward = 0
            episode_collisions = 0
            episode_speeds = []

            while not (done or truncated):
                # Predict action using the agent
                action, _states = agent.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = environment.step(action)

                # Update reward
                episode_reward += reward
                
                # Check for collisions
                if info.get("crashed", False):
                    episode_collisions += 1

                # Gather traffic speeds
                current_speeds = [
                    vehicle.speed
                    for vehicle in environment.unwrapped.road.vehicles  # Use unwrapped
                    if vehicle != environment.unwrapped.vehicle  # Exclude agent vehicle
                ]
                episode_speeds.extend(current_speeds)

                # Optional rendering
                environment.render()

            environment.close()

            # Collect data for this episode
            total_rewards.append(episode_reward)
            total_collisions += episode_collisions
            traffic_speeds.extend(episode_speeds)

            test_results.append({
                'episode': episode,
                'total_reward': episode_reward,
                'collisions': episode_collisions,
                'avg_speed': np.mean(episode_speeds) if episode_speeds else 0,
                'speed_variance': np.var(episode_speeds) if episode_speeds else 0
            })
        
        except Exception as e:
            logger.error(f"Test episode {episode + 1} failed in {environment_name}: {e}")

    # Calculate overall KPIs
    avg_reward = np.mean(total_rewards)
    avg_speed = np.mean(traffic_speeds) if traffic_speeds else 0
    speed_variance = np.var(traffic_speeds) if traffic_speeds else 0

    logger.info(f"Test completed in {environment_name} - "
                f"Avg Reward: {avg_reward}, Total Collisions: {total_collisions}, "
                f"Avg Traffic Speed: {avg_speed}, Speed Variance: {speed_variance}")

    # Save test results and KPIs
    save_pipeline_data(agent_name, stage, environment_name, {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        },
        'environment': environment_name,
        'stage': stage
    })

    return {
        'test_results': test_results,
        'kpis': {
            'average_reward': avg_reward,
            'total_collisions': total_collisions,
            'average_speed': avg_speed,
            'speed_variance': speed_variance
        }
    }

In [23]:
def rl_pipeline(initial_agent, agent_name, environments):
    """
    Sequential environment training pipeline with customizable order
    
    Args:
        initial_agent (sb3.BaseAlgorithm): Initial RL agent
        agent_name (str): Name of the agent
        environments (list of tuples): A list of (env_name, gym.Env) tuples defining the order of execution.
            Example: [("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)]
    
    Returns:
        Trained agent
    """

    logger.info("Customizable Sequential Environment Pipeline Started")
    
    current_agent = initial_agent

    for i, (env_name, env) in enumerate(environments):
        logger.info(f"Stage {i + 1}: {env_name.capitalize()} Environment")
        
        if i > 0:  # Test the previous agent in the current environment before training
            previous_env_name = environments[i - 1][0]
            logger.info(f"Testing {previous_env_name.capitalize()} Agent in {env_name.capitalize()}")
            test(
                current_agent, env, 
                agent_name, 'pre_training_test', env_name
            )
        
        # Train in the current environment
        logger.info(f"Training in {env_name.capitalize()} Environment")
        current_agent = train(
            current_agent, env, 
            agent_name, env_name, env_name, 20000
        )
        
        # Test in the current environment
        logger.info(f"Testing in {env_name.capitalize()} Environment")
        test(
            current_agent, env, 
            agent_name, 'pos_training_test', env_name
        )

    logger.info("Customizable Sequential Environment Pipeline Completed")

    # Save the final agent
    create_directory(f"models/{agent_name}/final")
    current_agent.save(f"models/{agent_name}/{agent_name}_final_model")
    
    return current_agent


In [24]:
# %tensorboard --logdir "agent_DQN"

- ### Agent 1

In [30]:
agent_DQN = DQN('MlpPolicy', roundabout_env,
        policy_kwargs=dict(net_arch=[256, 256]),
        learning_rate = 1e-4,
        buffer_size=15000,
        learning_starts=200,
        batch_size=32,
        gamma=0.8,
        train_freq=1,
        gradient_steps=1,
        target_update_interval=50,
        verbose=1,
        tensorboard_log="agent_DQN/")

final_agent_DQN = rl_pipeline(agent_DQN, 'agent_DQN', [("roundabout", roundabout_env)])#("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)])


""" Maybe compare to this! """

# # Training order is lane changing -> roundabout -> overtaking
# final_agent_DQN_lro = rl_pipeline(agent_DQN, 'agent_DQN_lro', [("lane_changing", lane_changing_env), ("roundabout", roundabout_env), ("overtaking", overtaking_env)])

# # Training order is roundabout -> overtaking -> lane changing
# final_agent_DQN_rol = rl_pipeline(agent_DQN, 'agent_DQN_rol', [("roundabout", roundabout_env), ("overtaking", overtaking_env), ("lane_changing", lane_changing_env)])

# # Training order is overtaking -> lane changing -> roundabout
# final_agent_DQN_olr = rl_pipeline(agent_DQN, 'agent_DQN_olr', [("overtaking", overtaking_env), ("lane_changing", lane_changing_env), ("roundabout", roundabout_env)])

# # Training order is overtaking -> roundabout -> lane changing
# final_agent_DQN_orl = rl_pipeline(agent_DQN, 'agent_DQN_orl', [("overtaking", overtaking_env), ("roundabout", roundabout_env), ("lane_changing", lane_changing_env)])

# # Training order is roundabout -> lane changing -> overtaking
# final_agent_DQN_rlo = rl_pipeline(agent_DQN, 'agent_DQN_rlo', [("roundabout", roundabout_env), ("lane_changing", lane_changing_env), ("overtaking", overtaking_env)])

# # Training order is lane changing -> overtaking -> roundabout
# final_agent_DQN_lor = rl_pipeline(agent_DQN, 'agent_DQN_lor', [("lane_changing", lane_changing_env), ("overtaking", overtaking_env), ("roundabout", roundabout_env)])

2024-12-12 10:31:31,797 - INFO - Customizable Sequential Environment Pipeline Started
2024-12-12 10:31:31,798 - INFO - Stage 1: Roundabout Environment
2024-12-12 10:31:31,798 - INFO - Training in Roundabout Environment


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a DummyVecEnv.
Logging to agent_DQN/DQN_10
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.5     |
|    ep_rew_mean      | 3.07     |
|    exploration_rate | 0.98     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 13       |
|    time_elapsed     | 3        |
|    total_timesteps  | 42       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.8     |
|    ep_rew_mean      | 2.15     |
|    exploration_rate | 0.959    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 13       |
|    time_elapsed     | 6        |
|    total_timesteps  | 86       |
----------------------------------
----------------------------------
| rollout/            |          |
| 

KeyboardInterrupt: 

- ### Agent N

## Performance

In [13]:
# Display results
def display_results(agent_name, stages, type="table"):
    # Load results
    results = {}
    for stage in stages:
        with open(f"results/{agent_name}/{stage}_data.json", 'r') as f:
            results[stage] = json.load(f)

    if type == "list":
        print(f"--- {agent_name} ---\n")
        for stage, data in results.items():
            print(f"> {stage}")
            print(f"    - Average Reward: {data['kpis']['average_reward']}")
            print(f"    - Total Collisions: {data['kpis']['total_collisions']}")
            print(f"    - Average Speed: {data['kpis']['average_speed']}")
            print(f"    - Speed Variance: {data['kpis']['speed_variance']}")
            print("")
        print("--------------------\n")
    elif type == "table":
        table = []
        for stage, data in results.items():
            table.append([stage, data['kpis']['average_reward'], data['kpis']['total_collisions'], data['kpis']['average_speed'], data['kpis']['speed_variance']])
        table = np.array(table)

        print("|".join([f"{{[{str(agent_name)}]:<20}} {str('Stage'):>10} ", "Average Reward".center(20), "Total Collisions".center(20), "Average Speed".center(20), "Speed Variance".center(20)]))
        print("=" * 117)
        for row in table:
            formatted_row = [
                f"{str(row[0]):>31} ",    # Stage (left aligned)
                f"{str(row[1]):^20}",    # Average Reward (center aligned)
                f"{str(row[2]):^20}",    # Total Collisions (center aligned)
                f"{str(row[3]):^20}",    # Average Speed (center aligned)
                f"{str(row[4]):^20}",     # Speed Variance (center aligned)
            ]
            print("|".join(formatted_row))
            print("-" * 117)
        print()
        

In [14]:
display_results("agent_DQN", ["lane_changing_pos_training_test", "roundabout_pre_training_test", "roundabout_pos_training_test", "overtaking_pre_training_test", "overtaking_pos_training_test"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
lane_changing_pos_training_test | 7.683333333333333  |         10         | 20.38586665050951  | 2.994659433616237  
---------------------------------------------------------------------------------------------------------------------
   roundabout_pre_training_test |        5.0         |         8          | 13.176681440042938 | 17.91196936083314  
---------------------------------------------------------------------------------------------------------------------
   roundabout_pos_training_test | 8.633333333333333  |         0          | 15.363874398994303 | 3.696733004851535  
---------------------------------------------------------------------------------------------------------------------
   overtaking_pre_training_test | 9.662684444444448  |         8          | 20.412763661430365 | 2.3286179404918492 
----------------------------------------------------------------

In [27]:
# Test the final agent in all environments
def test_agent_all_envs(agent_name, environments, environment_names):
    # Load the final agent
    agent = DQN.load(f"models/{agent_name}/{agent_name}_final_model", device="cpu")

    # Lane Changing
    performance_lane_changing = test(agent, environments[0], agent_name, '', "final_test_" + environment_names[0])

    # Roundabout
    performance_roundabout = test(agent, environments[1], agent_name, '', "final_test_" + environment_names[1])

    # Overtaking
    performance_overtaking = test(agent, environments[2], agent_name, '', "final_test_" + environment_names[2])


In [28]:
test_agent_all_envs("agent_DQN", [roundabout_env], ["roundabout"])#[lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


# test_agent_all_envs("agent_DQN_rol", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_olr", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_orl", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_rlo", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])
# test_agent_all_envs("agent_DQN_lor", [lane_changing_env, roundabout_env, overtaking_env], ["lane_changing", "roundabout", "overtaking"])


2024-12-12 10:30:48,203 - INFO - Test completed in final_test_roundabout - Avg Reward: 7.712499999999999, Total Collisions: 1, Avg Traffic Speed: 15.013559340080562, Speed Variance: 13.78392022820183
2024-12-12 10:30:48,204 - INFO - Directory created: results/agent_DQN
2024-12-12 10:30:48,205 - INFO - Data saved for  in final_test_roundabout


IndexError: list index out of range

## Analysis

In [20]:
# Display final test results
display_results("agent_DQN", ["final_test_lane_changing_", "final_test_roundabout_", "final_test_overtaking_"])

{[agent_DQN]:<20}      Stage |   Average Reward   |  Total Collisions  |   Average Speed    |   Speed Variance   
      final_test_lane_changing_ |      16.4125       |         4          | 20.602228832201057 | 1.7628795927127507 
---------------------------------------------------------------------------------------------------------------------
         final_test_roundabout_ |       12.05        |         0          | 14.381887780369855 | 19.82951714419477  
---------------------------------------------------------------------------------------------------------------------
         final_test_overtaking_ | 19.708888888888897 |         3          | 20.647436386620896 | 1.9794265486352098 
---------------------------------------------------------------------------------------------------------------------



## Conclusion